In [1]:
#r "nuget:Microsoft.ML,1.5.1"
#r "nuget:Microsoft.ML.AutoML,0.17.1"
#r "nuget:Microsoft.Data.Analysis,0.4.0"
#r "nuget:CsvHelper,16.0.0"

Installed package CsvHelper version 16.0.0

Installed package Microsoft.ML version 1.5.1

Installed package Microsoft.ML.AutoML version 0.17.1

Installed package Microsoft.Data.Analysis version 0.4.0

In [1]:
using Microsoft.Data.Analysis;
using XPlot.Plotly;

Add a formatter for `DataFrame` objects so that we can print the output as an html table.

In [1]:
using Microsoft.AspNetCore.Html;

Formatter.Register<DataFrame>((df, writer) =>
{
    var headers = new List<IHtmlContent>();
    headers.Add(th(i("index")));
    headers.AddRange(df.Columns.Select(c => (IHtmlContent) th(c.Name)));
    var rows = new List<List<IHtmlContent>>();
    var take = 20;
    for (var i = 0; i < Math.Min(take, df.Rows.Count); i++)
    {
        var cells = new List<IHtmlContent>();
        cells.Add(td(i));
        foreach (var obj in df.Rows[i])
        {
            cells.Add(td(obj));
        }
        rows.Add(cells);
    }
    
    var t = table(
        thead(
            headers),
        tbody(
            rows.Select(
                r => tr(r))));
    
    writer.Write(t);
}, "text/html");

In [1]:
using System.IO;
using System.Data;
using System.Globalization;
using CsvHelper;
using CsvHelper.Configuration;

static void NormalizeCSV(string inFile, string outFile)
{
    var culture = CultureInfo.InvariantCulture;

    using var reader = new StreamReader(inFile);
    using var csvIn = new CsvReader(reader, culture);
    using var recordsIn = new CsvDataReader(csvIn);
    using var writer = new StreamWriter(outFile);
    using var csvOut = new CsvWriter(writer, culture);    

    //change this strategy if you prefer to just delete the commas
    static string Sanitise(string s) => s.Replace(",", "_");

    var dataReader = new CsvDataReader(csvIn);
    var dt = new DataTable();
    dt.Load(dataReader);

    foreach (DataColumn col in dt.Columns)
    {
        csvOut.WriteField(col.ColumnName);
    }
    csvOut.NextRecord();

    foreach (DataRow row in dt.Rows)
    {
        for (var i = 0; i < dt.Columns.Count; i++)
        {
            csvOut.WriteField(Sanitise(row[i].ToString()));
        }

        csvOut.NextRecord();
    }
}

Read in the Housing Price Index CSV.

In [1]:
var housingDF = DataFrame.LoadCsv("HPImaster.csv", separator: ',', header: true);

Error: System.IO.FileNotFoundException: Could not find file 'c:\Users\matt.sheehan.APTERYX\Projects\Github\jupyter-notebooks-dotnet\housing\HPImaster.csv'.
File name: 'c:\Users\matt.sheehan.APTERYX\Projects\Github\jupyter-notebooks-dotnet\housing\HPImaster.csv'
   at System.IO.FileStream.ValidateFileHandle(SafeFileHandle fileHandle)
   at System.IO.FileStream.CreateFileOpenHandle(FileMode mode, FileShare share, FileOptions options)
   at System.IO.FileStream..ctor(String path, FileMode mode, FileAccess access, FileShare share, Int32 bufferSize, FileOptions options)
   at System.IO.FileStream..ctor(String path, FileMode mode)
   at Microsoft.Data.Analysis.DataFrame.LoadCsv(String filename, Char separator, Boolean header, String[] columnNames, Type[] dataTypes, Int32 numRows, Int32 guessRows, Boolean addIndexColumn, Encoding encoding)
   at Submission#10.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location where exception was thrown ---
   at Microsoft.Cod

In [1]:
housingDF = housingDF.Filter(housingDF.Columns["frequency"].ElementwiseEquals("quarterly"));
housingDF = housingDF.Filter(housingDF.Columns["place_name"].ElementwiseEquals("Ohio"));
housingDF = housingDF.Filter(housingDF.Columns["hpi_type"].ElementwiseEquals("traditional"));
housingDF = housingDF.Filter(housingDF.Columns["hpi_flavor"].ElementwiseEquals("all-transactions"));


housingDF.Columns.Add(new StringDataFrameColumn("yr_period", housingDF.Rows.Count()));

for (var i = 0; i < housingDF.Rows.Count(); i++)
{
    var yr = housingDF.Columns["yr"][i].ToString();
    var period = housingDF.Columns["period"][i].ToString();

    housingDF.Columns["yr_period"][i] = DateTime.Parse(yr + "-" + period).ToString("yyyy-MM");
}

housingDF

Error: System.NullReferenceException: Object reference not set to an instance of an object.
   at Submission#11.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location where exception was thrown ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

Graph the final data.

In [1]:
var h = new Graph.Scatter()
{
    x = housingDF.Columns["yr_period"],
    y = housingDF.Columns["index_nsa"],
    mode = "lines"
};
var chart = Chart.Plot(h);
chart.WithLabels(new [] {"Ohio" });
chart.WithLegend(true);
chart.WithYTitle("Housing Cost");

display(chart);

Error: System.NullReferenceException: Object reference not set to an instance of an object.
   at Submission#12.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location where exception was thrown ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)